In [1]:
import torch
import numpy as np
from transformers import AutoTokenizer, Trainer
from datasets import Dataset

In [2]:
from loading import load_test
from preprocessing import preprocess
from submission import prepare_submission
from bert import tokenize, get_BERT

[nltk_data] Error loading stopwords: <urlopen error [Errno 104]
[nltk_data]     Connection reset by peer>
[nltk_data] Downloading package wordnet to /home/jonhue/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
MODEL = 'test1'
TOKENIZER = 'bert-base-uncased'
PREPROCESSING = None

In [5]:
df_test = load_test(x_col='text')
preprocess(df_test, flags=PREPROCESSING)
dataset_test = Dataset.from_pandas(df_test)

In [6]:
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER)
test_tokenized = tokenize(dataset_test, tokenizer, path=f'bert/cache/test_tokenized__{TOKENIZER}')

Parameter 'function'=<function tokenize.<locals>.tokenize_function at 0x7fedf19df790> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
100%|██████████| 10/10 [00:01<00:00,  6.60ba/s]


In [7]:
model = get_BERT(MODEL, device)

In [8]:
trainer = Trainer(model, tokenizer=tokenizer)

In [9]:
test_pred = trainer.predict(test_tokenized)
test_pred = np.argmax(test_pred.predictions, axis=1)

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: __index_level_0__, text, token_type_ids. If __index_level_0__, text, token_type_ids are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 10000
  Batch size = 8


In [10]:
prepare_submission(test_pred)